In [12]:
#preprecess imports

import pandas as pd
import spacy
import numpy as np
import en_core_web_sm
# nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()
from tqdm.notebook import tqdm 

#

In [5]:
!pip show spacy

In [13]:
df = pd.read_csv('practice.csv')

In [16]:
df = df.dropna(axis='rows')

In [17]:
df

,Unnamed: 0,pub_type,position_section,position_subsection,hl1,hl2,author,lede,body,language,word_count,copyright,content-id,volume,issue_number,edition,pub_name,pub_date,licensor_indexing_terms,indexing_terms
1,1,"Newspaper, Newspapers",SPORTS,Football-Patriots,Wilfork goes out a legend,Lineman leaves Pats for Texans,JEFF HOWE,Vince Wilfork will be immortalized as a Patrio...,'Few players reached or will ever reach the sp...,['ENGLISH'],576,Copyright 2015 Boston Herald Inc. All Rights R...,BHERLD-25185125,Vol. 33,No. 76,All Editions,The Boston Herald,17-03-2015,"WILFORK, PATRIOTS, DEFENSIVE, COACH, LINEMAN, ...","{'legal': [{'className': 'Education Law', 'cla..."
2,2,"Newspaper, Newspapers",OPINION,Commentary,Hillary's modus operandi not pretty,'Nasty' GOP latest excuse for private email use,JENNIFER C. BRACERAS,Just when you thought we were done with emailg...,Didn't Clinton tell us that she set up the pri...,['ENGLISH'],642,Copyright 2015 Boston Herald Inc. All Rights R...,BHERLD-25223487,Vol. 33,No. 76,All Editions,The Boston Herald,17-03-2015,"CLINTON, PRIVATE, EMAILS, CARVILLE, SERVER, AC...","{'subject': [{'score': '90', 'classCode': 'N81..."
6,6,"Newspaper, Newspapers",BUSINESS,Retail,RETAILERS FEAR TRASHED STREETS,Mess left beneath snow piles could slow return...,MARIE SZANISZLO,With dirty snowbanks still lining many streets...,'After this very tough winter for main streets...,['ENGLISH'],412,Copyright 2015 Boston Herald Inc. All Rights R...,BHERLD-25226117,Vol. 33,No. 76,All Editions,The Boston Herald,17-03-2015,"APRIL, BOSTON, CONSUMERS, MAYOR, SNOW, SPRING,...","{'legal': [{'className': 'Governments', 'class..."
7,7,"Newspaper, Newspapers",FEATURES,Television-Television Reviews,Night of the laughing dead,"CW's 'iZombie' mixes horror, mystery, comedy",MARK A. PERIGARD,It's the 'Walking Dead' 'Quincy'/'Psych' mash...,"One night, the super-overachiever decides to t...",['ENGLISH'],502,Copyright 2015 Boston Herald Inc. All Rights R...,BHERLD-25223152,Vol. 33,No. 76,All Editions,The Boston Herald,17-03-2015,"IZOMBIE, LIV, DEAD, BRAINS, HORROR, KOHLI, MCI...","{'subject': [{'score': '89', 'classCode': 'Z3V..."
8,8,"Newspaper, Newspapers",SPORTS,Baseball-Red Sox,Sox see what's missing,Harvey shows off top-of-rotation arm,JASON MASTRODONATO,"FORT MYERS - It's not just the results, the co...","The Red Sox don't have a Matt Harvey, and they...",['ENGLISH'],564,Copyright 2015 Boston Herald Inc. All Rights R...,BHERLD-25185113,Vol. 33,No. 76,All Editions,The Boston Herald,17-03-2015,"HARVEY, SOX, RED, ACE, INNING, MATT, PITCHERS,...","{'subject': [{'score': '90', 'classCode': 'STX..."
14,14,"Newspaper, Newspapers",NEWS,Local,Cops give riveting firefight details,Shed light on a.m. battle with Tsarnaevs,LAUREL J. SWEET,The showdown with the bomb-tossing Tsarnaev br...,Watertown police Sgts. Jeffrey Pugliese and Jo...,['ENGLISH'],467,Copyright 2015 Boston Herald Inc. All Rights R...,BHERLD-25225885,Vol. 33,No. 76,All Editions,The Boston Herald,17-03-2015,"TAMERLAN, PUGLIESE, DZHOKHAR, POLICE, REYNOLDS...","{'subject': [{'score': '90', 'classCode': 'ST0..."
17,17,"Newspaper, Newspapers",BUSINESS,Economy,Make 'Madness' work at work,Have fun and staff will go through hoops,JUDITH BOWMAN,The phenomenon known as 'March Madness' has of...,Employees across the nation are sure to take t...,['ENGLISH'],384,Copyright 2015 Boston Herald Inc. All Rights R...,BHERLD-25228025,Vol. 33,No. 76,All Editions,The Boston Herald,17-03-2015,"JUDITH, BOWMAN, MADNESS, RULES, MARCH, OPPORTU...","{'subject': [{'score': '90', 'classCode': 'ST0..."
21,21,"Newspaper, Newspapers",SPORTS,Basketball-Celtics,Right where C's belong,Win streak hits five as foes take notice,MARK MURPHY,The Celtics' reputation continued to shine las...,'I think they've improved a lot defensively si...,['ENGLISH'],572,Copyright 2015 Boston Herald Inc. All Rights R...,BHERLD-25231940,Vol. 33,No. 76,All Editions,The Boston Herald,17-03-2015,"CELTICS, ZELLER, SEASON, BRADLEY, WIN, GAME, P

In [18]:
def combine_led_body(df):
    body = df['body'].values
    lede = df['lede'].values
    arr = np.vstack((lede,body)).T
    for step, x in enumerate(arr): 
        arr[step] = ' '.join(x)

    arr = arr.flatten()
    return arr 

In [19]:
corpus = combine_led_body(df)

In [20]:
def custom_preprocess(corpus_iterable): 
    return [[z.lemma_.lower() for z in y if not (z.is_stop or z.text == '\n')] for y in 
tqdm(nlp.pipe(corpus_iterable), total = len(corpus_iterable)) ]

In [21]:
pre_processed_corpus = custom_preprocess(corpus)

  0%|          | 0/30 [00:00<?, ?it/s]

In [23]:
pre_processed_corpus

zsh:1: command not found: pre_processed_corpus
